In [1]:
import numpy as np
import sympy

sympy.init_printing()

In [2]:
A, B, C, D, E, F, G, H = sympy.symbols("A B C D E F G H")

In [3]:
from tqdm import tqdm

pd_matrices = []
non_pd_matrices = []

for i in tqdm(range(1000)):
    # A, E, G > 0
    A_val, E_val, G_val = np.random.uniform(0, 5, size=3)

    # sample imaginary parts
    B_val, C_val, D_val, F_val, H_val = np.random.uniform(-5, 5, size=5)

    g_mat_1 = sympy.Matrix([
        [A_val + sympy.I * B_val, C_val + sympy.I * D_val, 0],
        [C_val + sympy.I * D_val, E_val + sympy.I * F_val, 0],
        [0, 0, G_val + sympy.I * H_val]
    ])

    if A_val * E_val > C_val ** 2:
        # it needs to satisfy AE > C^2
        pd_matrices.append(g_mat_1)
    else:
        non_pd_matrices.append(g_mat_1)

100%|██████████| 1000/1000 [00:00<00:00, 1104.68it/s]


In [4]:
test_matrix = pd_matrices[45]
test_matrix

⎡0.969529403830848 - 3.15322384102035⋅ⅈ   1.47070736648915 - 2.1108683950897⋅ⅈ
⎢                                                                             
⎢ 1.47070736648915 - 2.1108683950897⋅ⅈ   3.02177355690116 - 0.944637015920325⋅
⎢                                                                             
⎣                  0                                       0                  

                     0                   ⎤
                                         ⎥
ⅈ                    0                   ⎥
                                         ⎥
   0.675268010078879 - 2.08845532070875⋅ⅈ⎦

In [5]:
A_val, B_val = test_matrix[0, 0].args
C_val, D_val = test_matrix[0, 1].args
E_val, F_val = test_matrix[1, 1].args
G_val, H_val = test_matrix[2, 2].args

B_val = B_val.args[0]
D_val = D_val.args[0]
F_val = F_val.args[0]
H_val = H_val.args[0]

val_dict = {A: A_val, B: B_val, C: C_val, D: D_val, E: E_val, F: F_val, G: G_val, H: H_val}
val_dict

In [6]:
kappa, beta = sympy.symbols("kappa beta")
kappa, beta

In [7]:
g_mat = sympy.Matrix([
    [kappa * (A + sympy.I * B), kappa * (C + sympy.I * D), 0],
    [kappa * (C + sympy.I * D), kappa * (E + sympy.I * F), 0],
    [0, 0, beta * (G + sympy.I * H)]
])
g_mat

⎡κ⋅(A + ⅈ⋅B)  κ⋅(C + ⅈ⋅D)       0     ⎤
⎢                                     ⎥
⎢κ⋅(C + ⅈ⋅D)  κ⋅(E + ⅈ⋅F)       0     ⎥
⎢                                     ⎥
⎣     0            0       β⋅(G + ⅈ⋅H)⎦

In [65]:
g_mat_real = sympy.Matrix([[kappa * A, kappa * C, 0], [kappa * C, kappa * E, 0], [0, 0, beta * G]])
g_mat_real

⎡A⋅κ  C⋅κ   0 ⎤
⎢             ⎥
⎢C⋅κ  E⋅κ   0 ⎥
⎢             ⎥
⎣ 0    0   G⋅β⎦

### Diagonalize Real Part

In [66]:
eigenvects_real_sym = g_mat_real.eigenvects()
eigenvects_real_sym

⎡                 ⎛                                              ⎡⎡           
⎢                 ⎜                                              ⎢⎢           
⎢                 ⎜                 ________________________     ⎢⎢      κ⋅(A 
⎢⎛        ⎡⎡0⎤⎤⎞  ⎜                ╱  2              2    2      ⎢⎢      ─────
⎢⎜        ⎢⎢ ⎥⎥⎟  ⎜κ⋅(A + E)   κ⋅╲╱  A  - 2⋅A⋅E + 4⋅C  + E       ⎢⎢  E       2
⎢⎜G⋅β, 1, ⎢⎢0⎥⎥⎟, ⎜───────── - ─────────────────────────────, 1, ⎢⎢- ─ + ─────
⎢⎜        ⎢⎢ ⎥⎥⎟  ⎜    2                     2                   ⎢⎢  C        
⎢⎝        ⎣⎣1⎦⎦⎠  ⎜                                              ⎢⎢           
⎢                 ⎜                                              ⎢⎢           
⎢                 ⎜                                              ⎢⎢           
⎣                 ⎝                                              ⎣⎣           

            ________________________⎤⎤⎞  ⎛                                    
           ╱  2              2    2 ⎥⎥⎟  ⎜         

#### Eigenvectors

In [67]:
eig1 = eigenvects_real_sym[2][2][0]
eig1 /= sympy.sqrt(eig1.dot(eig1))

eig2 = eigenvects_real_sym[1][2][0]
eig2 /= sympy.sqrt(eig2.dot(eig2))

eig3 = eigenvects_real_sym[0][2][0]
eig3 /= sympy.sqrt(eig3.dot(eig3))

In [69]:
Q_sym = sympy.Matrix.hstack(eig1, eig2, eig3)
Q_sym.simplify()
Q_sym

⎡                      ________________________                               
⎢                     ╱  2              2    2                                
⎢           A - E + ╲╱  A  - 2⋅A⋅E + 4⋅C  + E                           A - E 
⎢──────────────────────────────────────────────────────────  ─────────────────
⎢          ________________________________________________            _______
⎢         ╱                                              2            ╱       
⎢        ╱         ⎛            ________________________⎞            ╱        
⎢       ╱      2   ⎜           ╱  2              2    2 ⎟           ╱      2  
⎢      ╱    4⋅C  + ⎝-A + E - ╲╱  A  - 2⋅A⋅E + 4⋅C  + E  ⎠          ╱    4⋅C  +
⎢C⋅   ╱     ──────────────────────────────────────────────   C⋅   ╱     ──────
⎢    ╱                             2                             ╱            
⎢  ╲╱                             C                            ╲╱             
⎢                                                   

In [12]:
Q_sym.subs(val_dict)

⎡0.462493628200829  -0.886622605099618  0⎤
⎢                                        ⎥
⎢0.886622605099618  0.462493628200829   0⎥
⎢                                        ⎥
⎣        0                  0           1⎦

In [13]:
eigenvects_real_num = sympy.re(test_matrix).eigenvects()
eigenvects_real_num

⎡⎛                      ⎡⎡-0.886622605099618⎤⎤⎞  ⎛                     ⎡⎡-0.46
⎢⎜                      ⎢⎢                  ⎥⎥⎟  ⎜                     ⎢⎢     
⎢⎜0.202356559334257, 1, ⎢⎢0.462493628200829 ⎥⎥⎟, ⎜3.78894640139775, 1, ⎢⎢-0.88
⎢⎜                      ⎢⎢                  ⎥⎥⎟  ⎜                     ⎢⎢     
⎣⎝                      ⎣⎣        0         ⎦⎦⎠  ⎝                     ⎣⎣     

2493628200829⎤⎤⎞  ⎛                      ⎡⎡ 0 ⎤⎤⎞⎤
             ⎥⎥⎟  ⎜                      ⎢⎢   ⎥⎥⎟⎥
6622605099618⎥⎥⎟, ⎜0.675268010078879, 1, ⎢⎢ 0 ⎥⎥⎟⎥
             ⎥⎥⎟  ⎜                      ⎢⎢   ⎥⎥⎟⎥
   0         ⎦⎦⎠  ⎝                      ⎣⎣1.0⎦⎦⎠⎦

In [14]:
Q_num = sympy.Matrix.hstack(eigenvects_real_num[0][2][0], eigenvects_real_num[1][2][0], eigenvects_real_num[2][2][0])
Q_num

⎡-0.886622605099618  -0.462493628200829   0 ⎤
⎢                                           ⎥
⎢0.462493628200829   -0.886622605099618   0 ⎥
⎢                                           ⎥
⎣        0                   0           1.0⎦

#### Eigenvalues

In [15]:
lmda_mat_sym = sympy.Matrix([[eigenvects_real_sym[2][0], 0, 0], [0, eigenvects_real_sym[1][0], 0], [0, 0, eigenvects_real_sym[0][0]]])
lmda_mat_sym

⎡                 ________________________                                    
⎢                ╱  2              2    2                                     
⎢κ⋅(A + E)   κ⋅╲╱  A  - 2⋅A⋅E + 4⋅C  + E                                      
⎢───────── + ─────────────────────────────                      0             
⎢    2                     2                                                  
⎢                                                                             
⎢                                                            _________________
⎢                                                           ╱  2              
⎢                                           κ⋅(A + E)   κ⋅╲╱  A  - 2⋅A⋅E + 4⋅C
⎢                    0                      ───────── - ──────────────────────
⎢                                               2                     2       
⎢                                                                             
⎣                    0                              

In [16]:
lmda_mat_sym.subs(val_dict)

⎡3.78894640139775⋅κ           0                    0         ⎤
⎢                                                            ⎥
⎢        0           0.202356559334257⋅κ           0         ⎥
⎢                                                            ⎥
⎣        0                    0           0.675268010078879⋅β⎦

In [17]:
lmda_mat_num = sympy.Matrix([[eigenvects_real_num[0][0], 0, 0], [0, eigenvects_real_num[1][0], 0], [0, 0, eigenvects_real_num[2][0]]])
lmda_mat_num

⎡0.202356559334257         0                  0        ⎤
⎢                                                      ⎥
⎢        0          3.78894640139775          0        ⎥
⎢                                                      ⎥
⎣        0                 0          0.675268010078879⎦

#### Inverse square root of eigenvalues

In [18]:
P_sym = lmda_mat_sym.inv().applyfunc(sympy.sqrt)
P_sym.simplify()
P_sym

⎡         _________________________________________                           
⎢        ╱                    1                                               
⎢√2⋅    ╱  ───────────────────────────────────────                           0
⎢      ╱     ⎛           ________________________⎞                            
⎢     ╱      ⎜          ╱  2              2    2 ⎟                            
⎢   ╲╱     κ⋅⎝A + E + ╲╱  A  - 2⋅A⋅E + 4⋅C  + E  ⎠                            
⎢                                                                             
⎢                                                             ________________
⎢                                                            ╱                
⎢                        0                           √2⋅    ╱  ───────────────
⎢                                                          ╱     ⎛           _
⎢                                                         ╱      ⎜          ╱ 
⎢                                                   

In [19]:
P_sym.subs(val_dict).evalf()

⎡                     0.5                                                  ⎤
⎢                  ⎛1⎞                                                     ⎥
⎢0.513736909881192⋅⎜─⎟                0                        0           ⎥
⎢                  ⎝κ⎠                                                     ⎥
⎢                                                                          ⎥
⎢                                              0.5                         ⎥
⎢                                           ⎛1⎞                            ⎥
⎢           0              2.22300969500845⋅⎜─⎟                0           ⎥
⎢                                           ⎝κ⎠                            ⎥
⎢                                                                          ⎥
⎢                                                                       0.5⎥
⎢                                                                    ⎛1⎞   ⎥
⎢           0                         0             1.21691967270067⋅⎜─⎟   ⎥

In [20]:
P_num = lmda_mat_num.inv().applyfunc(sympy.sqrt)
P_num

⎡2.22300969500845          0                 0        ⎤
⎢                                                     ⎥
⎢       0          0.513736909881192         0        ⎥
⎢                                                     ⎥
⎣       0                  0          1.21691967270067⎦

The real part has become the identity.

In [78]:
Z_sym = Q_sym @ P_sym
Z_sym.simplify()
Z_sym

⎡         _________________________________________ ⎛           ______________
⎢        ╱                    1                     ⎜          ╱  2           
⎢√2⋅    ╱  ─────────────────────────────────────── ⋅⎝A - E + ╲╱  A  - 2⋅A⋅E + 
⎢      ╱     ⎛           ________________________⎞                            
⎢     ╱      ⎜          ╱  2              2    2 ⎟                            
⎢   ╲╱     κ⋅⎝A + E + ╲╱  A  - 2⋅A⋅E + 4⋅C  + E  ⎠                            
⎢─────────────────────────────────────────────────────────────────────────────
⎢                         _______________________________________________     
⎢                        ╱                                             2      
⎢                       ╱         ⎛           ________________________⎞       
⎢                      ╱      2   ⎜          ╱  2              2    2 ⎟       
⎢                     ╱    4⋅C  + ⎝A - E + ╲╱  A  - 2⋅A⋅E + 4⋅C  + E  ⎠       
⎢               C⋅   ╱     ─────────────────────────

In [22]:
Z_sym.subs(val_dict).evalf()

⎡                     0.5                      0.5                         ⎤
⎢                  ⎛1⎞                      ⎛1⎞                            ⎥
⎢0.237600047391635⋅⎜─⎟     -1.9709706469501⋅⎜─⎟                0           ⎥
⎢                  ⎝κ⎠                      ⎝κ⎠                            ⎥
⎢                                                                          ⎥
⎢                    0.5                       0.5                         ⎥
⎢                 ⎛1⎞                       ⎛1⎞                            ⎥
⎢0.45549075737469⋅⎜─⎟      1.02812781937008⋅⎜─⎟                0           ⎥
⎢                 ⎝κ⎠                       ⎝κ⎠                            ⎥
⎢                                                                          ⎥
⎢                                                                       0.5⎥
⎢                                                                    ⎛1⎞   ⎥
⎢           0                         0             1.21691967270067⋅⎜─⎟   ⎥

In [23]:
(Z_sym.T @ g_mat_real @ Z_sym).subs(val_dict)

⎡        1.0⋅κ                     ⎤
⎢        ─────             0     0 ⎥
⎢          κ                       ⎥
⎢                                  ⎥
⎢1.38777878078145e-16⋅κ  1.0⋅κ     ⎥
⎢──────────────────────  ─────   0 ⎥
⎢          κ               κ       ⎥
⎢                                  ⎥
⎣          0               0    1.0⎦

In [24]:
Z_num = Q_num @ P_num
Z_num

⎡-1.9709706469501  -0.237600047391635         0        ⎤
⎢                                                      ⎥
⎢1.02812781937008  -0.45549075737469          0        ⎥
⎢                                                      ⎥
⎣       0                  0           1.21691967270067⎦

In [25]:
(Q_num @ P_num).T @ sympy.re(test_matrix) @ (Q_num @ P_num)

⎡1.0  -1.2490009027033e-16   0 ⎤
⎢                              ⎥
⎢ 0           1.0            0 ⎥
⎢                              ⎥
⎣ 0            0            1.0⎦

### Diagonalize Imaginary Part

#### Transform imaginary part

In [79]:
g_mat_imag = sympy.Matrix([[kappa * B, kappa * D, 0], [kappa * D, kappa * F, 0], [0, 0, beta * H]])
g_mat_imag

⎡B⋅κ  D⋅κ   0 ⎤
⎢             ⎥
⎢D⋅κ  F⋅κ   0 ⎥
⎢             ⎥
⎣ 0    0   H⋅β⎦

In [81]:
g_mat_imag_trans_sym = Z_sym.T @ g_mat_imag @ Z_sym
g_mat_imag_trans_sym.simplify()
g_mat_imag_trans_sym

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [28]:
g_mat_imag_trans_sym.subs(val_dict).evalf()

⎡-0.830892866628987   2.4136884053234           0        ⎤
⎢                                                        ⎥
⎢ 2.4136884053234    -4.69296534347106          0        ⎥
⎢                                                        ⎥
⎣        0                   0          -3.09277988818809⎦

In [29]:
g_mat_imag_trans_num = Z_num.T @ sympy.im(test_matrix) @ Z_num
g_mat_imag_trans_num

⎡-4.69296534347106   -2.4136884053234           0        ⎤
⎢                                                        ⎥
⎢-2.4136884053234   -0.830892866628987          0        ⎥
⎢                                                        ⎥
⎣        0                  0           -3.09277988818809⎦

#### Eigenvectors

In [43]:
eigenvects_imag_sym = g_mat_imag_trans_sym.eigenvects()
eigenvects_imag_sym

⎡               ⎛                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                       ______________________________________
⎢⎛      ⎡⎡0⎤⎤⎞  ⎜                      ╱  2  2                                
⎢⎜H     ⎢⎢ ⎥⎥⎟  ⎜A⋅F + B⋅E - 2⋅C⋅D   ╲╱  A ⋅F  - 2⋅A⋅B⋅E⋅F - 4⋅A⋅C⋅D⋅F + 4⋅A⋅D
⎢⎜─, 1, ⎢⎢0⎥⎥⎟, ⎜───────────────── - ─────────────────────────────────────────
⎢⎜G     ⎢⎢ ⎥⎥⎟  ⎜     ⎛       2⎞                                       ⎛      
⎢⎝      ⎣⎣1⎦⎦⎠  ⎜   2⋅⎝A⋅E - C ⎠                    

In [31]:
eig_imag_1 = eigenvects_imag_sym[2][2][0]
eig_imag_1 /= sympy.sqrt(eig_imag_1.dot(eig_imag_1))

eig_imag_2 = eigenvects_imag_sym[1][2][0]
eig_imag_2 /= sympy.sqrt(eig_imag_2.dot(eig_imag_2))

eig_imag_3 = eigenvects_imag_sym[0][2][0]
eig_imag_3 /= sympy.sqrt(eig_imag_3.dot(eig_imag_3))

In [32]:
Q2_sym = sympy.Matrix.hstack(eig_imag_1, eig_imag_2, eig_imag_3)

In [33]:
Q2_sym.subs(val_dict).evalf()

⎡0.901307915249923  -0.43317899522924   0 ⎤
⎢                                         ⎥
⎢0.433178995229237  0.901307915249936   0 ⎥
⎢                                         ⎥
⎣        0                  0          1.0⎦

In [34]:
eigenvects_imag_num = g_mat_imag_trans_num.eigenvects()
eigenvects_imag_num

⎡⎛                      ⎡⎡0.901307915249936⎤⎤⎞  ⎛                      ⎡⎡-0.43
⎢⎜                      ⎢⎢                 ⎥⎥⎟  ⎜                      ⎢⎢     
⎢⎜-5.85301187199236, 1, ⎢⎢0.43317899522924 ⎥⎥⎟, ⎜0.329153661892314, 1, ⎢⎢0.901
⎢⎜                      ⎢⎢                 ⎥⎥⎟  ⎜                      ⎢⎢     
⎣⎝                      ⎣⎣        0        ⎦⎦⎠  ⎝                      ⎣⎣     

3178995229241⎤⎤⎞  ⎛                      ⎡⎡ 0 ⎤⎤⎞⎤
             ⎥⎥⎟  ⎜                      ⎢⎢   ⎥⎥⎟⎥
307915249936 ⎥⎥⎟, ⎜-3.09277988818809, 1, ⎢⎢ 0 ⎥⎥⎟⎥
             ⎥⎥⎟  ⎜                      ⎢⎢   ⎥⎥⎟⎥
   0         ⎦⎦⎠  ⎝                      ⎣⎣1.0⎦⎦⎠⎦

In [35]:
Q2_num = sympy.Matrix.hstack(eigenvects_imag_num[1][2][0], eigenvects_imag_num[0][2][0], eigenvects_imag_num[2][2][0])
Q2_num

⎡-0.433178995229241  0.901307915249936   0 ⎤
⎢                                          ⎥
⎢0.901307915249936   0.43317899522924    0 ⎥
⎢                                          ⎥
⎣        0                   0          1.0⎦

#### Eigenvalues

In [83]:
lmda_mat_sym_2 = sympy.Matrix([[eigenvects_imag_sym[2][0], 0, 0], [0, eigenvects_imag_sym[1][0], 0], [0, 0, eigenvects_imag_sym[0][0]]])
lmda_mat_sym_2

⎡                       ______________________________________________________
⎢                      ╱  2  2                                2      2  2     
⎢A⋅F + B⋅E - 2⋅C⋅D   ╲╱  A ⋅F  - 2⋅A⋅B⋅E⋅F - 4⋅A⋅C⋅D⋅F + 4⋅A⋅D ⋅E + B ⋅E  + 4⋅
⎢───────────────── + ─────────────────────────────────────────────────────────
⎢     ⎛       2⎞                                       ⎛       2⎞             
⎢   2⋅⎝A⋅E - C ⎠                                     2⋅⎝A⋅E - C ⎠             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                               0                             
⎢                                                                             
⎢                                                   

In [48]:
lmda_mat_sym_2[0, 0].args[0].args[2].args[0]

In [49]:
((A * F - B * E) ** 2 + 4 * ((C * F - D * E) * (B * C - A * D)) - lmda_mat_sym_2[0, 0].args[0].args[2].args[0]).simplify()

In [50]:
(A * F - B * E) ** 2 + 4 * ((C * F - D * E) * (B * C - A * D))

In [51]:
gamma_val = A * E * H ** 2 + A * F * G * H + B * E * G * H - B * F * H ** 2 + D ** 2 * G ** 2 + D ** 2 * H ** 2
gamma_val

In [52]:
for i in tqdm(range(10000)):
    # A, E, G > 0
    A_val, E_val, G_val = np.random.uniform(0, 5, size=3)

    # sample imaginary parts
    B_val, C_val, D_val, F_val, H_val = np.random.uniform(-5, 5, size=5)

    gamma_val_i = gamma_val.subs({A: A_val, B: B_val, D: D_val, E: E_val, F: F_val, G: G_val, H: H_val}).evalf()
    if gamma_val_i < 0:
        continue

    C_val = ((-D_val * G_val - sympy.sqrt(gamma_val_i)) / H_val).evalf()

    disc = ((A * F - B * E) ** 2 + 4 * ((C * F - D * E) * (B * C - A * D))).subs({A: A_val, B: B_val, C: C_val, D: D_val, E: E_val, F: F_val})
    if disc < 0:
        print(disc)
        break

  0%|          | 2/10000 [00:00<03:22, 49.32it/s]

-803.169865307829


In [53]:
((A * F - B * E) ** 2 + 4 * ((C * F - D * E) * (B * C - A * D)) - (A * F + B * E - 2 * C * D) ** 2).simplify().factor()

In [54]:
(A * F + B * E - 2 * C * D) ** 2 - 4 * (A * E - C ** 2) * (B * F - D ** 2)

In [55]:
lmda_mat_sym_2.subs(val_dict)

⎡0.329153661892313          0                  0        ⎤
⎢                                                       ⎥
⎢        0          -5.85301187199236          0        ⎥
⎢                                                       ⎥
⎣        0                  0          -3.09277988818809⎦

In [56]:
lmda_mat_num_2 = sympy.Matrix([[eigenvects_imag_num[1][0], 0, 0], [0, eigenvects_imag_num[0][0], 0], [0, 0, eigenvects_imag_num[2][0]]])
lmda_mat_num_2

⎡0.329153661892314          0                  0        ⎤
⎢                                                       ⎥
⎢        0          -5.85301187199236          0        ⎥
⎢                                                       ⎥
⎣        0                  0          -3.09277988818809⎦

### Final Transformation

In [57]:
T_sym = Q_sym @ P_sym @ Q2_sym

In [58]:
T_sym.subs(val_dict).evalf()

⎡                      0.5                       0.5                         ⎤
⎢                   ⎛1⎞                       ⎛1⎞                            ⎥
⎢-0.639632281094324⋅⎜─⎟     -1.87937479461694⋅⎜─⎟                0           ⎥
⎢                   ⎝κ⎠                       ⎝κ⎠                            ⎥
⎢                                                                            ⎥
⎢                     0.5                        0.5                         ⎥
⎢                  ⎛1⎞                        ⎛1⎞                            ⎥
⎢0.855900800706946⋅⎜─⎟      0.729350712871131⋅⎜─⎟                0           ⎥
⎢                  ⎝κ⎠                        ⎝κ⎠                            ⎥
⎢                                                                            ⎥
⎢                                                                         0.5⎥
⎢                                                                      ⎛1⎞   ⎥
⎢            0                         0            

In [59]:
(T_sym.T @ g_mat @ T_sym).subs(val_dict).evalf()

⎡                    ⎛                                                        
⎢                    ⎜                                                        
⎢- 0.639632281094324⋅⎜0.855900800706946⋅κ⋅(1.47070736648915 - 2.1108683950897⋅
⎢                    ⎝                                                        
⎢                                                                             
⎢                     ⎛                                                       
⎢                     ⎜                                                       
⎢ - 0.639632281094324⋅⎜0.729350712871131⋅κ⋅(1.47070736648915 - 2.1108683950897
⎢                     ⎝                                                       
⎢                                                                             
⎣                                                                             

      0.5                                                                   0.
   ⎛1⎞                                             

In [60]:
T_num = Q_num @ P_num @ Q2_num
T_num

⎡0.639632281094329   -1.87937479461694         0        ⎤
⎢                                                       ⎥
⎢-0.855900800706956  0.729350712871132         0        ⎥
⎢                                                       ⎥
⎣        0                   0          1.21691967270067⎦

In [61]:
T_num.T @ test_matrix @ T_num

⎡          1.0 + 0.329153661892314⋅ⅈ              -7.7715611723761e-16⋅ⅈ      
⎢                                                                             
⎢-2.22044604925031e-16 - 4.44089209850063e-16⋅ⅈ  1.0 - 5.85301187199236⋅ⅈ     
⎢                                                                             
⎣                      0                                    0              1.0

        0            ⎤
                     ⎥
        0            ⎥
                     ⎥
 - 3.09277988818809⋅ⅈ⎦